https://towardsdatascience.com/rnn-recurrent-neural-networks-how-to-successfully-model-sequential-data-in-python-5a0b9e494f92

In [15]:
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense, SimpleRNN # for creating regular densely-connected NN layers and RNN layers

# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version
import math # to help with data reshaping of the data

# Sklearn
import sklearn # for model evaluation
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.model_selection import train_test_split # for splitting the data into train and test samples
from sklearn.metrics import mean_squared_error # for model evaluation metrics
from sklearn.preprocessing import MinMaxScaler # for feature scaling

# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version

Tensorflow/Keras: 2.12.0
pandas: 1.5.2
numpy: 1.22.3
sklearn: 1.2.0
plotly: 5.13.1


In [16]:
# Set Pandas options to display more columns
pd.options.display.max_columns=100

# Read in the eeg data csv
df=pd.read_csv('eeg_data.csv', encoding='utf-8')

# Show a snaphsot of data
df

,video_id,subject_id,EEG.AF3,EEG.F7,EEG.F3,EEG.FC5,EEG.T7,EEG.P7,EEG.O1,EEG.O2,EEG.P8,EEG.T8,EEG.FC6,EEG.F4,EEG.F8,EEG.AF4,POW.AF3.Theta,POW.AF3.Alpha,POW.AF3.BetaL,POW.AF3.BetaH,POW.AF3.Gamma,POW.F7.Theta,POW.F7.Alpha,POW.F7.BetaL,POW.F7.BetaH,POW.F7.Gamma,POW.F3.Theta,POW.F3.Alpha,POW.F3.BetaL,POW.F3.BetaH,POW.F3.Gamma,POW.FC5.Theta,POW.FC5.Alpha,POW.FC5.BetaL,POW.FC5.BetaH,POW.FC5.Gamma,POW.T7.Theta,POW.T7.Alpha,POW.T7.BetaL,POW.T7.BetaH,POW.T7.Gamma,POW.P7.Theta,POW.P7.Alpha,POW.P7.BetaL,POW.P7.BetaH,POW.P7.Gamma,POW.O1.Theta,POW.O1.Alpha,POW.O1.BetaL,POW.O1.BetaH,POW.O1.Gamma,POW.O2.Theta,POW.O2.Alpha,POW.O2.BetaL,POW.O2.BetaH,POW.O2.Gamma,POW.P8.Theta,POW.P8.Alpha,POW.P8.BetaL,POW.P8.BetaH,POW.P8.Gamma,POW.T8.Theta,POW.T8.Alpha,POW.T8.BetaL,POW.T8.BetaH,POW.T8.Gamma,POW.FC6.Theta,POW.FC6.Alpha,POW.FC6.BetaL,POW.FC6.BetaH,POW.FC6.Gamma,POW.F4.Theta,POW.F4.Alpha,POW.F4.BetaL,POW.F4.BetaH,POW.F4.Gamma,POW.F8.Theta,POW.F8.Alpha,POW.F8.BetaL,POW.F8.BetaH,POW.F8.Gamma,POW.AF4.Theta,POW.AF4.Alpha,POW.AF4.BetaL,POW.AF4.BetaH,POW.AF4.Gamma,subject_understood
0,0,0,4210.641113,4179.102539,4287.948730,4235.384766,4207.948730,4165.000000,4135.897461,4170.000000,4155.384766,4157.179688,4610.384766,4388.846191,4413.461426,4499.743652,1.859064,1.488007,0.294396,0.459987,0.188314,1.779598,0.620659,0.497043,0.344509,0.178338,1.840744,1.740009,0.369845,0.484037,0.187594,1.842673,0.856627,0.474424,0.415397,0.175614,0.584676,1.208085,0.214748,0.144198,0.120753,1.297408,0.761949,0.354184,0.166404,0.099637,1.422160,0.889572,0.552382,0.149173,0.136558,2.078273,1.472955,1.057647,0.280043,0.153327,1.409152,0.979982,0.869081,0.417896,0.188206,1.428878,0.877251,0.553363,0.383072,0.112488,2.348271,1.782157,0.644852,0.520060,0.154410,2.349918,2.376053,0.655009,0.613906,1.120513,3.758370,1.583895,0.504567,0.471979,0.138717,1.801014,1.504794,0.258570,0.435745,0.469483,0
1,0,0,4201.025879,4188.717773,4280.128418,4236.922852,4209.615234,4152.436035,4130.128418,4149.487305,4149.487305,4157.820313,4583.717773,4376.666504,4392.820313,4488.461426,2.030504,1.509949,0.496135,0.452138,0.194018,1.821380,0.596975,0.685146,0.319841,0.174923,1.941082,1.797096,0.558355,0.485693,0.189281,1.945797,0.943827,0.654867,0.437179,0.175975,0.641788,1.284735,0.212188,0.161631,0.120083,1.522173,0.821546,0.295174,0.191005,0.110549,1.635566,0.813352,0.416753,0.149681,0.127509,2.409505,1.458509,1.079635,0.326943,0.152629,1.462499,0.907250,0.969058,0.482167,0.192056,1.761167,0.858840,0.714015,0.438183,0.102417,2.969623,1.850320,0.841046,0.601471,0.139706,2.478265,2.495719,0.745543,0.663186,1.769841,4.580270,1.709560,0.606587,0.527616,0.155580,1.859177,1.379617,0.317579,0.468416,0.642560,0
2,0,0,4203.205078,4182.820313,4282.820313,4231.025879,4207.820313,4172.436035,4131.538574,4147.948730,4131.666504,4131.666504,4574.743652,4377.051270,4390.512695,4483.077148,2.315652,1.541873,0.812744,0.455024,0.193780,1.897375,0.612869,0.982626,0.304586,0.164330,2.164562,1.862794,0.852138,0.483437,0.184689,2.115402,1.042002,0.946591,0.449115,0.172039,0.705399,1.248802,0.220355,0.173320,0.116159,1.822336,0.855601,0.262621,0.212594,0.118272,1.789751,0.748843,0.348561,0.153822,0.115047,2.633745,1.418934,1.177704,0.363531,0.152218,1.576797,0.863319,1.054493,0.520667,0.193345,2.099791,0.900051,0.864675,0.452751,0.094539,3.570862,1.931157,1.099680,0.659150,0.131624,2.612426,2.580640,0.909882,0.681112,2.377539,5.230789,1.873591,0.795834,0.565414,0.170816,2.027946,1.283876,0.441925,0.494701,0.798197,0
3,0,0,4186.538574,4168.717773,4266.794922,4229.230957,4202.179688,4155.384766,4128.333496,4151.666504,4136.666504,4134.230957,4565.512695,4362.436035,4380.897461,4471.025879,2.713728,1.648073,1.184512,0.466466,0.189903,2.000522,0.716119,1.314321,0.311415,0.150169,2.531722,1.999752,1.189854,0.474792,0.176145,2.327959,1.185726,1.291802,0.455347,0.166402,0.761616,1.111277,0.243083,0.176220,0.110383,2.241397,0.836288,0.256094,0.225850,0.121525,1.868363,0.700736,0.350339,0.160516,0.101635,2.665952,1.371016,1.2

In [17]:
def prep_data(datain, time_step):
    # 1. y-array  
    # First, create an array with indices for y elements based on the chosen time_step
    y_indices = np.arange(start=time_step, stop=len(datain), step=time_step)
    # Create y array based on the above indices 
    y_tmp = datain[y_indices]
    
    # 2. X-array  
    # We want to have the same number of rows for X as we do for y
    rows_X = len(y_tmp)
    # Since the last element in y_tmp may not be the last element of the datain, 
    # let's ensure that X array stops with the last y
    X_tmp = datain[range(time_step*rows_X)]
    # Now take this array and reshape it into the desired shape
    X_tmp = np.reshape(X_tmp, (rows_X, time_step, 1))
    return X_tmp, y_tmp

In [18]:
#Read dataset
data = pd.read_csv("EEG_data.csv")

data = data.dropna(axis=0)
for col in data.columns:
  print(col)

video_id
subject_id
EEG.AF3
EEG.F7
EEG.F3
EEG.FC5
EEG.T7
EEG.P7
EEG.O1
EEG.O2
EEG.P8
EEG.T8
EEG.FC6
EEG.F4
EEG.F8
EEG.AF4
POW.AF3.Theta
POW.AF3.Alpha
POW.AF3.BetaL
POW.AF3.BetaH
POW.AF3.Gamma
POW.F7.Theta
POW.F7.Alpha
POW.F7.BetaL
POW.F7.BetaH
POW.F7.Gamma
POW.F3.Theta
POW.F3.Alpha
POW.F3.BetaL
POW.F3.BetaH
POW.F3.Gamma
POW.FC5.Theta
POW.FC5.Alpha
POW.FC5.BetaL
POW.FC5.BetaH
POW.FC5.Gamma
POW.T7.Theta
POW.T7.Alpha
POW.T7.BetaL
POW.T7.BetaH
POW.T7.Gamma
POW.P7.Theta
POW.P7.Alpha
POW.P7.BetaL
POW.P7.BetaH
POW.P7.Gamma
POW.O1.Theta
POW.O1.Alpha
POW.O1.BetaL
POW.O1.BetaH
POW.O1.Gamma
POW.O2.Theta
POW.O2.Alpha
POW.O2.BetaL
POW.O2.BetaH
POW.O2.Gamma
POW.P8.Theta
POW.P8.Alpha
POW.P8.BetaL
POW.P8.BetaH
POW.P8.Gamma
POW.T8.Theta
POW.T8.Alpha
POW.T8.BetaL
POW.T8.BetaH
POW.T8.Gamma
POW.FC6.Theta
POW.FC6.Alpha
POW.FC6.BetaL
POW.FC6.BetaH
POW.FC6.Gamma
POW.F4.Theta
POW.F4.Alpha
POW.F4.BetaL
POW.F4.BetaH
POW.F4.Gamma
POW.F8.Theta
POW.F8.Alpha
POW.F8.BetaL
POW.F8.BetaH
POW.F8.Gamma
POW.AF4.Theta
POW.

In [19]:
alpha_cols =[]
beta_cols=[]
gamma_cols=[]
theta_cols=[]

channels=["F7", 'F3', 'P7', 'O1', 'O2', 'P8','AF4']

for col in data.columns:
  col_list=list(col.split("."))
  # print(col_list)
  if (len(col_list)==3):
    if ("Alpha" in col_list[2] and col_list[1] in channels):
      alpha_cols.append(col)
    elif ("Beta" in col_list[2] and col_list[1] in channels):
      beta_cols.append(col)
    elif ("Gamma" in col_list[2] and col_list[1] in channels):
      gamma_cols.append(col)
    elif ("Theta" in col_list[2] and col_list[1] in channels):
      theta_cols.append(col)
print(len(alpha_cols),alpha_cols)
print(len(beta_cols),beta_cols)
print(len(gamma_cols),gamma_cols)
print(len(theta_cols),theta_cols)

7 ['POW.F7.Alpha', 'POW.F3.Alpha', 'POW.P7.Alpha', 'POW.O1.Alpha', 'POW.O2.Alpha', 'POW.P8.Alpha', 'POW.AF4.Alpha']
14 ['POW.F7.BetaL', 'POW.F7.BetaH', 'POW.F3.BetaL', 'POW.F3.BetaH', 'POW.P7.BetaL', 'POW.P7.BetaH', 'POW.O1.BetaL', 'POW.O1.BetaH', 'POW.O2.BetaL', 'POW.O2.BetaH', 'POW.P8.BetaL', 'POW.P8.BetaH', 'POW.AF4.BetaL', 'POW.AF4.BetaH']
7 ['POW.F7.Gamma', 'POW.F3.Gamma', 'POW.P7.Gamma', 'POW.O1.Gamma', 'POW.O2.Gamma', 'POW.P8.Gamma', 'POW.AF4.Gamma']
7 ['POW.F7.Theta', 'POW.F3.Theta', 'POW.P7.Theta', 'POW.O1.Theta', 'POW.O2.Theta', 'POW.P8.Theta', 'POW.AF4.Theta']


In [20]:
alpha=data[alpha_cols].mean(axis=1)
beta=data[beta_cols].mean(axis=1)
gamma=data[gamma_cols].mean(axis=1)
theta=data[theta_cols].mean(axis=1)
print(alpha,beta,gamma,theta)

0        1.138560
1        1.110621
2        1.092319
3        1.115628
4        1.207377
           ...   
68826    1.637583
68827    1.439577
68828    1.285126
68829    1.182737
68830    1.117769
Length: 68831, dtype: float64 0        0.445469
1        0.481818
2        0.546672
3        0.622338
4        0.689747
           ...   
68826    0.911338
68827    0.889141
68828    0.840476
68829    0.766783
68830    0.686598
Length: 68831, dtype: float64 0        0.201878
1        0.227072
2        0.246585
3        0.254398
4        0.246751
           ...   
68826    0.456128
68827    0.473429
68828    0.489906
68829    0.498479
68830    0.493219
Length: 68831, dtype: float64 0        1.661193
1        1.807340
2        1.987502
3        2.183097
4        2.344865
           ...   
68826    0.985212
68827    1.127806
68828    1.322875
68829    1.531517
68830    1.684475
Length: 68831, dtype: float64


In [21]:
#ABTR= BETA/(APHA+THETA)
#AGR = APHA/GAMMA
#TBR = THETA/BETA

abtr = []
agr = []
tbr = []

for (a,b,g,t) in zip(alpha,beta,gamma,theta):
  feature_abtr = b/(a+t)
  abtr.append(feature_abtr)

  feature_agr = a/g
  agr.append(feature_agr)

  feature_tbr = t/b
  tbr.append(feature_tbr)

In [22]:
df = pd.DataFrame(list(zip(abtr, agr,tbr)),columns =['abtr', 'agr','tbr'])
print(df)

           abtr       agr       tbr
0      0.159110  5.639854  3.729091
1      0.165121  4.891042  3.751088
2      0.177501  4.429781  3.635639
3      0.188660  4.385371  3.507896
4      0.194172  4.893092  3.399601
...         ...       ...       ...
68826  0.347468  3.590186  1.081061
68827  0.346322  3.040747  1.268422
68828  0.322268  2.623210  1.573960
68829  0.282502  2.372693  1.997327
68830  0.245017  2.266273  2.453364

[68831 rows x 3 columns]


In [23]:
#x_int holds all the input that will train the machine learning model
x_int = df

# #y_int contains the output/label to indicate whether the subject understood the lecture or not 
y_int = data["subject_understood"]

# y_int = y_int.dropna(axis=0)
# x_int = x_int.dropna(axis=0)
# y_int = y_int.values
# x_int = x_int.values

# #Using 80% of data to train the model and 20% of the data to test the model
x_train, x_test, y_train, y_test = train_test_split(x_int, y_int, test_size=0.2,random_state=42)
print('x train',x_train,'\n x test', x_test,'\n y train', y_train,'\n y test', y_test)

x train            abtr        agr        tbr
56390  0.099961   7.313723   6.453630
24912  0.093007  14.474470   7.356275
24822  0.071271   8.326097  10.602472
44236  0.094514  13.288529   6.874982
45248  0.295440   2.178452   2.445405
...         ...        ...        ...
37194  0.101508   5.145293   7.377970
6265   0.140753   7.720524   4.387119
54886  0.159160   2.780806   3.890348
860    0.253000  13.462422   1.492623
15795  0.054422  13.184494  15.426566

[55064 rows x 3 columns] 
 x test            abtr       agr        tbr
54873  0.110897  1.846611   6.063296
20237  0.105888  3.823915   4.119782
14792  0.092837  4.366782   7.610283
61165  0.263019  2.244861   2.808181
18846  0.215169  4.710587   3.213026
...         ...       ...        ...
39844  0.070484  6.202890   9.031889
45314  0.232904  2.064100   2.960710
19260  0.176346  3.128294   4.232881
10646  0.039609  6.277639  22.739621
18994  0.267726  5.010608   2.534767

[13767 rows x 3 columns] 
 y train 56390    1
24912    1

In [24]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(55064, 3)
(55064,)
(13767, 3)
(13767,)


In [25]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN

# Define the model
model = Sequential()
model.add(SimpleRNN(64, input_shape=(3, 1)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

x_train=x_train.to_numpy()
y_train=y_train.to_numpy()
x_test=x_test.to_numpy()
y_test=y_test.to_numpy()
# Train the model
x_train = x_train.reshape((55064, 3, 1))
model.fit(x_train, y_train, epochs=80, batch_size=64)

# Evaluate the model
x_test = x_test.reshape((13767, 3, 1))
loss, accuracy = model.evaluate(x_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Epoch 1/80
861/861 [==============================] - 4s 3ms/step - loss: 0.4398 - accuracy: 0.8173
Epoch 2/80
861/861 [==============================] - 2s 2ms/step - loss: 0.4314 - accuracy: 0.8197
Epoch 3/80
861/861 [==============================] - 2s 2ms/step - loss: 0.4288 - accuracy: 0.8210
Epoch 4/80
861/861 [==============================] - 2s 2ms/step - loss: 0.4277 - accuracy: 0.8206
Epoch 5/80
861/861 [==============================] - 2s 2ms/step - loss: 0.4259 - accuracy: 0.8204
Epoch 6/80
861/861 [==============================] - 2s 2ms/step - loss: 0.4255 - accuracy: 0.8209
Epoch 7/80
861/861 [==============================] - 3s 3ms/step - loss: 0.4248 - accuracy: 0.8208
Epoch 8/80
861/861 [==============================] - 2s 3ms/step - loss: 0.4236 - accuracy: 0.8211
Epoch 9/80
861/861 [==============================] - 2s 2ms/step - loss: 0.4224 - accuracy: 0.8214
Epoch 10/80
861/861 [==============================] - 2s 3ms/step - loss: 0.4224 - accuracy: 0.8216